In [ ]:
import numpy as np
import pickle
# !pip install bcolz
import bcolz

In [ ]:
from collections import Counter
from functools import reduce
# from tqdm.notebook import tqdm
import string
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
import torch
import urllib
import glob
import os
import numpy as np
import pandas as pd
import torch.nn as nn
import nltk
from PIL import Image
from torchvision import transforms
import torchvision
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet

In [ ]:
from PIL import Image
import pandas as pd
import spacy
from torch.nn.utils.rnn import pad_sequence
spacy_eng = spacy.load("en_core_web_sm")

In [ ]:
def read_img(file):
    img_id=[]
    for img in glob.glob(file+'*.jpg'):
        temp=os.path.basename(img)
        img_id.append(temp)
        
    return img_id

In [ ]:
# extract the data from tar files

In [1]:
# use glove
glove_path = '/scratch/varunc/glove/'
glove_zip = glove_path+'glove.6B.zip'

In [ ]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1


# Need not run this again and again


words = []
idx = 0
word2idx = {}
vectors = bcolz.carray(np.zeros(1), rootdir=glove_path+'6B.300.dat', mode='w')

with open(glove_path+'glove.6B.300d.txt', 'rb') as f:
    pbar = tqdm(total=file_len(glove_path+'glove.6B.300d.txt'))
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)
        pbar.update(1)
    
vectors = bcolz.carray(vectors[1:].reshape((400000, 300)), rootdir=glove_path+'6B.300.dat', mode='w')
vectors.flush()
pickle.dump(words, open(glove_path+'6B.300_words.pkl', 'wb'))
pickle.dump(word2idx, open(glove_path+'6B.300_idx.pkl', 'wb'))

In [ ]:
vectors = bcolz.open(glove_path+'6B.300.dat')[:]
words = pickle.load(open(glove_path+'6B.300_words.pkl', 'rb'))
word2idx = pickle.load(open(glove_path+'6B.300_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

In [ ]:
data_imgid=read_img(flickr_path+"Flickr-8K/Flicker8k_Dataset/")


In [ ]:
class Vocabulary:
    def __init__(self, freq_thresh):
        self.freq_thresh = freq_thresh
        self.itos = {0:"<PAD>", 1:"<SOS>", 2:"<EOS>", 3:"<UNK>"}
        self.stoi = {"<PAD>":0, "<SOS>":1, "<EOS>":2, "<UNK>":3}
    
    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenizer(text):
        return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

    def build_vocab(self, sentence_list):
        lemmatizer = WordNetLemmatizer()
        frequencies = {}
        idx = 4
        for sentence in sentence_list:
            for word in self.tokenizer(sentence):
                lw = lemmatizer.lemmatize(word)
                if word not in frequencies:
                    frequencies[word] = 1
                else:
                    frequencies[word] += 1
                if frequencies[word] == self.freq_thresh:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1
                    
                if lw not in frequencies:
                    frequencies[lw] = 1
                else:
                    frequencies[lw] += 1
                if frequencies[lw] == self.freq_thresh:
                    self.stoi[lw] = idx
                    self.itos[idx] = lw
                    idx += 1
    
    def numericalize(self, text):
        tok_text = self.tokenizer(text)
        vec = [self.stoi[word] if word in self.stoi else self.stoi["<UNK>"] for word in tok_text]
        return vec

In [ ]:
# for test and train we need to create new text files of the format (Flickr8k.token.txt)
class FlickrDataset(Dataset):
    def __init__(self, root_dir, caption_file, lemma_file, img_file, freq_thresh, transform=None,vocab = None, lemma_vocab=None):
        self.freq_thresh = freq_thresh
        self.transform = transform
        self.root_dir = root_dir
        self.img_file = img_file
        self.caption_file = caption_file
        self.lemma_file = lemma_file
        
        self.caption_dict = self.imgId_caption_dict()
        self.lemma_dict = self.imgId_lemma_dict()
        
        self.imgs , self.captions, self.all_captions = self.load_img_caption()
        if vocab is None:
            self.vocab = Vocabulary(self.freq_thresh)
            self.vocab.build_vocab(self.captions)
        else:
            self.vocab = vocab
            
        self.imgs_lemma , self.lemmas = self.load_img_lemma()
        if lemma_vocab is None:
            self.lemma_vocab = Vocabulary(self.freq_thresh)
            self.lemma_vocab.build_vocab(self.lemmas)
        else:
            self.lemma_vocab = lemma_vocab

    def imgId_caption_dict(self):
        caption_dict = {}
        with open(self.caption_file, 'r') as f:
            for line in f.readlines():
                line = line.strip('\n')
                temp = line.split()
                img_name, _ = temp[0].split('#')  # first word will be img_id
                description = " ".join([ word for word in temp[1:]]) # get back the description
                if img_name not in caption_dict:
                    caption_dict[img_name] = [description]
                else:
                    caption_dict[img_name].append(description)
        return caption_dict

    def imgId_lemma_dict(self):
        stop_word=list(set(stopwords.words('english')))
        lemma_dict = {}
        with open(self.lemma_file, 'r') as f:
            for line in f.readlines():
                line = line.strip('\n')
                temp = line.split()
                img_name, _ = temp[0].split('#')  # first word will be img_id
                description = " ".join([ word.lower() for word in temp[1:] if word.lower() not in stop_word]) # get back the description
                description = description.translate(str.maketrans('', '', string.punctuation)).strip()
                if img_name not in lemma_dict:
                    lemma_dict[img_name] = [description]
                else:
                    lemma_dict[img_name].append(description)
        return lemma_dict

    def load_img_caption(self):
        imgs = []
        captions = []
        all_captions = []
        with open(self.img_file, 'r') as f:
            for line in f.readlines():
                line = line.strip('\n')
                for caption in self.caption_dict[line]:
                    imgs.append(line)
                    captions.append(caption)
                    all_captions.append(self.caption_dict[line])
        return imgs, captions, all_captions

    def load_img_lemma(self):
        imgs = []
        captions = []
        with open(self.img_file, 'r') as f:
            for line in f.readlines():
                line = line.strip('\n')
                for caption in self.lemma_dict[line]:
                    imgs.append(line)
                    captions.append(caption)
        return imgs, captions

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, index):
        caption = self.captions[index]
        all_caption = self.all_captions[index]
        img_id = self.imgs[index]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")

        if self.transform is not None:
            img = self.transform(img)

        numericalized_caption = [self.vocab.stoi["<SOS>"]]
        numericalized_caption += self.vocab.numericalize(caption)
        numericalized_caption += [self.vocab.stoi["<EOS>"]]
        
        all_num_captions = []
        for caption_i in all_caption:
            num_cap = [self.vocab.stoi["<SOS>"]]
            num_cap += self.vocab.numericalize(caption_i)
            num_cap += [self.vocab.stoi["<EOS>"]]
            all_num_captions += num_cap

        return img, torch.tensor(numericalized_caption), torch.tensor(all_num_captions)

In [3]:
# we can also define simply a function instead of a class
class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx
    
    def __call__(self, batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs, dim=0) # batch * imgs_size * 3 (RGB)
        captions = [item[1] for item in batch] 
        captions = pad_sequence(captions, batch_first=True, padding_value=self.pad_idx)
        all_captions = [item[2] for item in batch]
        all_captions = pad_sequence(all_captions, batch_first=True, padding_value=self.pad_idx)        
        return imgs, captions, all_captions # return the batched images and captions

In [4]:
def get_loader(root_dir, caption_file, lemma_file, img_file, transform, vocab=None, lemma_vocab=None, batch=32, num_worker=2, shuffle=True, pin_memory=True):
    dataset = FlickrDataset(root_dir=root_dir, caption_file=caption_file, lemma_file=lemma_file, img_file=img_file, freq_thresh=2, transform=transform,vocab=vocab, lemma_vocab=lemma_vocab)
    pad_idx = dataset.vocab.stoi["<PAD>"]
    loader = DataLoader(dataset=dataset,
                        batch_size=batch,
                        shuffle=shuffle,
                        collate_fn=MyCollate(pad_idx),
                        pin_memory=pin_memory,
                        num_workers=num_worker)
    return dataset,loader

In [ ]:
trans = transforms.Compose([
                            transforms.Resize((224, 224)),
                            transforms.ToTensor(),
])
dataset, dataloader = get_loader(root_dir=flickr_path+"Flickr-8K/Flicker8k_Dataset",
                        img_file=flickr_path+"Flickr-8K/Flickr_8k.trainImages.txt",
                        caption_file=flickr_path+"Flickr-8K/Flickr8k.token.txt",
                        lemma_file=flickr_path+"Flickr-8K/Flickr8k.lemma.token.txt",
                        transform=trans, batch=512,num_worker=30,
                        shuffle=True)

In [ ]:
test_dataset, test_dataloader = get_loader(root_dir=flickr_path+"Flickr-8K/Flicker8k_Dataset",
                        img_file=flickr_path+"Flickr-8K/Flickr_8k.testImages.txt",
                        caption_file=flickr_path+"Flickr-8K/Flickr8k.token.txt",
                        lemma_file=flickr_path+"Flickr-8K/Flickr8k.lemma.token.txt",
                        transform=trans,
                        vocab=dataset.vocab,
                        lemma_vocab=dataset.lemma_vocab,
                        batch=1, shuffle=True)

## GoogleNet

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class EncoderCNN(nn.Module):
    def __init__(self):
        super(EncoderCNN, self).__init__()
        googlenet = torch.hub.load('pytorch/vision:v0.9.0', 'googlenet', pretrained=True)
        for param in googlenet.parameters():
            param.requires_grad_(False)

        modules = list(googlenet.children())[:-2]
        self.googlenet = nn.Sequential(*modules)
    def forward(self, X):
        x = self.googlenet(X)
        x = x.view(x.shape[0],-1)
        return x

In [ ]:
model = nn.DataParallel(EncoderCNN())
model.eval()
model.to(device)

In [ ]:
def out_vect(filename):
    input_image = Image.open(filename)
    input_tensor = trans(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    input_batch = input_batch.to(device)

    with torch.no_grad():
        output = model(input_batch)
        
    return output

In [ ]:
def out_vect_tensor(input_tensor):
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    input_batch = input_batch.to(device)

    with torch.no_grad():
        output = model(input_batch)
        
    return output

In [ ]:
def get_vector(file,directory):
    img_vector=[]
    imgs_list = []
    with open(file,'r') as f:
        for line in f.readlines():
            x=out_vect(os.path.join(directory,line.strip()))
            img_vector.append(x)
            imgs_list.append(line.strip())
    return imgs_vector, imgs_list

In [ ]:
imgs_vector, imgs_list=get_vector(flickr_path+'Flickr-8K/Flickr_8k.trainImages.txt',
                                  flickr_path+"Flickr-8K/Flicker8k_Dataset/")
imgs_list = np.array(imgs_list)
imgs_tensor = torch.cat(imgs_vector,0)

In [ ]:
def knn(data, test, num=5):
    dist = torch.norm(data - test, dim=1, p=2)
    knn = dist.topk(num, largest=False)
    
    return knn.indices.cpu().numpy()

In [ ]:
def get_words_ll(img_list):
    return [ dataset.lemma_dict[x] for x in img_list]

In [ ]:
def get_attr_encoded(vect, nums=10, train=True):
    if train:
        words_pref_ll = get_words_ll(imgs_list[knn(attr_tensor,vect,nums+1)[1:]])
    else:
        words_pref_ll = get_words_ll(imgs_list[knn(attr_tensor,vect,nums)])
    all_words = ' '.join(reduce(lambda x,y: x+y,words_pref_ll)).split()
    all_words = [ x for x in all_words if x in dataset.vocab.stoi]
    top_words = [ x for x,y in Counter(all_words).most_common(nums)]
    return top_words

In [ ]:
def get_attr_tensor(img_tensor,train=True):
    model.to(device)
    img_tensor = img_tensor.to(device)
    with torch.no_grad():
        vect = model(img_tensor)
    return get_attr_encoded(vect, train)